# Long Run Analysis

In [ ]:
from see import DataDownload as dd
imagefiles = []
maskfiles = []
outputfiles = []

print("Adding COSKEL Data")
files = dd.getCOSKELFolderlists(folder='./Image_Data/', outputfolder='./Image_Data/Output/')
imagefiles += files[0]
maskfiles += files[1]
outputfiles += files[2]

print("Adding Sky Data")
files = dd.getSkyFolderLists(folder='./Image_Data/', outputfolder='./Image_Data/Output/')
imagefiles += files[0]
maskfiles += files[1]
outputfiles += files[2]

folder = 'Image_Data/CO-SKEL-SKY/'
outputfile = "allre_Sky.npy"
#Sky Data
datarange = list(range(408))[-60:]

In [ ]:
from see import DataDownload as dd

#KOMATSUNA_output 
prefix = 20
folder = 'Image_Data/KOMATSUNA_output/'
outputfile = "allre_KOMATSUNA.npy"

imagefiles = []
maskfiles = []
outputfiles = []

print("Adding KOMATSUNA Data")
files = dd.getKomatsunaFolderLists(folder='./Image_Data/', outputfolder='./Image_Data/KOMATSUNA_output/')
imagefiles += files[0]
maskfiles += files[1]
outputfiles += files[2]

folder = 'Image_Data/KOMATSUNA_output/'
datarange = list(range(len(imagefiles)))

In [ ]:
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np
from pathlib import Path

def evaluate_run(prefix = 0, folder='./'):
    p = Path(folder)
    filefilter = f'{prefix}_*.output'
    print(filefilter)
    filenames = list(p.glob(filefilter))

    assert(len(filenames) > 0)

    fitness = []
    pop = []
    dist = []
    for filename in filenames:
        print(filename)
        fid  = open(filename, 'r')
        lines = fid.readlines() 
        for line in lines:
            if 'BEST' in line:
                fitness.append(eval(line.split('-')[1]))
                pop.append(eval(line.split(' - ')[2]))
            if 'HIST' in line:
                dist.append(eval(line.split('-')[1]))

    fitness = np.array(fitness)
    fitness = list(fitness[:,0])
    fitness.sort(reverse=True)
    plt.plot(fitness)
    plt.xlabel('Generation')
    plt.ylabel('Fitness')
    plt.savefig(f'{filename}.png')
    return fitness, pop, dist

In [ ]:
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np
from pathlib import Path

import imageio
from see import Segmentors
from see import JupyterGUI


allresults = []

for index in datarange:
    #print(f"{index} - {imagefiles[index]}")
    fitness_list, pop, dist = evaluate_run(index, folder=folder)
    params = pop[-1]
    seg = Segmentors.algoFromParams(params)
    results = []
    for test in datarange:
        img = imageio.imread(imagefiles[test])
        gmask = imageio.imread(maskfiles[test])
        mask = seg.evaluate(img)
        fitness,_,_ = Segmentors.FitnessFunction(mask,gmask)
        results.append(fitness)
    allresults.append(results)


In [ ]:
allre = np.array(allresults)
plt.imshow(allre)
plt.colorbar()

In [ ]:
np.save(outputfile, allre) # save
allre = np.load(outputfile) # loa

In [ ]:
np.sum(allre < 0.1)

In [ ]:
total = np.sum(allre < 0.1 ,axis=1)
total

In [ ]:
index_min = np.argmax(total)
index_min

In [ ]:
best = np.max(total)
best

In [ ]:
plt.plot(np.sum(allre, axis=1))